In [5]:
import snap
import json
import re
import os
import sys
import argparse
import bz2

In [6]:
top = 1000
def filter_subreddits(subreddit_metagraph):
    mg = snap.TNEANet.Load(snap.TFIn(subreddit_metagraph))
    srids_to_names = {}

    # Get the largest |top| subreddits by subscriber count
    acceptable = sorted(((NI.GetId(), mg.GetIntAttrDatN(NI, 'subscribers')) for NI in mg.Nodes()),
                        key=(lambda (nid, subs): subs), reverse=True)[:top]
    acceptable = set(nid for (nid, subs) in acceptable)

    for NI in mg.Nodes():
        # t5_xxxxx -> politics
        if NI.GetId() in acceptable:
            srids_to_names[mg.GetStrAttrDatN(NI, 'name')] = mg.GetStrAttrDatN(NI, 'url')[3:-1]

    return srids_to_names

In [9]:
subreddit_metagraph = 'subreddits.graph'
srids_to_srnames = filter_subreddits(subreddit_metagraph)

In [13]:
def parse_submissions(srids_to_srnames, submission_directory, outsubname):
    outfile = open(outsubname, 'w')
    outfile.write('\t'.join(['Author', 'Subreddit', 'Score', 'Gilded', 'Created_UTC', 'Is_Self',
                           'Title', 'Selftext']) + '\n')

    postids_to_authors = {}

    subfiles = []
    for fname in os.listdir(submission_directory):
        if fname.startswith('.') or fname.endswith('~'):
            continue
        path = os.path.join(submission_directory, fname)
        if os.path.isdir(path): # skip subdirectories
            continue
        subfiles.append(path)
        
    print('Parsing submissions...')
    for subfile in subfiles:
        print(subfile + ': parsing...')
        submissions = (json.loads(line) for line in bz2.BZ2File(subfile))
        for (i, sub) in enumerate(submissions):
            if sub['subreddit_id'] in srids_to_srnames: # subreddit_id is t5_xxxxx
                post_id = 't3_' + sub['id']
                author = sub['author'].lower()
                subreddit = srids_to_srnames[sub['subreddit_id']]
                score = sub['score']
                gold = sub.get('gilded', 0)
                timestamp = sub['created_utc']
                is_self = int(sub['is_self'])
                title = re.sub(r'\s+', ' ', sub['title'].encode('ascii', 'backslashreplace')).lower()
                selftext = '' if not is_self \
                           else re.sub(r'\s+', ' ', sub['selftext'].encode('ascii', 'backslashreplace')).lower()

                postids_to_authors[post_id] = author
                outfile.write('\t'.join(str(x) for x in [post_id, author, subreddit, score, gold, timestamp,
                                                         is_self, title, selftext]) + '\n')

            if i % 100000 == 0:
                print(i)
                
    outfile.close()

    return postids_to_authors

In [14]:
submission_directory = 'submissions_jan2012'
output_submission_text_file = 'reddit_submissions_jan2012.txt'
postids_to_authors = parse_submissions(srids_to_srnames, submission_directory, output_submission_text_file)

Parsing submissions...
submissions_jan2012/RS_2012-01.bz2: parsing...
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000


In [24]:
def parse_comments(srids_to_srnames, postids_to_authors, comment_directory, outcomname):
    outfile = open(outcomname, 'w')
    outfile.write('#Commenter\tCommentee\tSubreddit\tPost_ID\tControversiality\tUpvotes\t' + 
                  'Downvotes\tScore\tGilded\tCreated_UTC\tIs_Reply\tAuthor_Deleted\t' + 
                  'Text_Deleted\tComment_Text\n')
    
    comids_to_authors = {} # t1_xxxxx -> gallowboob, e.g.
        
    comfiles = []
    for fname in os.listdir(comment_directory):
        if fname.startswith('.') or fname.endswith('~'):
            continue
        path = os.path.join(comment_directory, fname)
        if os.path.isdir(path): # skip subdirectories
            continue
        comfiles.append(path)
        
    print('Parsing comments...')
    for comfile in comfiles:        
        print(comfile + ': parsing...')
        comments = (json.loads(line) for line in bz2.BZ2File(comfile))
        for (i, com) in enumerate(comments):
            sr_id = com['subreddit_id'] # t5_xxxxx
            post_id = com['link_id'] # t3_xxxxx
            parent_id = com['parent_id'] # t1_xxxxx (comment) if reply; t3_xxxxx (post) if top-level comment
            is_reply = parent_id.startswith('t1')
            
            if sr_id in srids_to_srnames and post_id in postids_to_authors\
            and (not is_reply or parent_id in comids_to_authors):
                sr_name = srids_to_srnames[sr_id]
                commenter = com['author'].lower()
                parent_dict = comids_to_authors if is_reply else postids_to_authors
                commentee = parent_dict[parent_id]
                controversiality = com['controversiality']
                upvotes = com['ups']
                downvotes = com['downs']
                score = com['score']
                gilded = com['gilded']
                created = com['created_utc']
                body = re.sub(r"\s+", ' ', com['body'].encode('ascii', 'backslashreplace')).lower()
                author_deleted = int(commenter == '[deleted]')
                text_deleted = int(body in ('[deleted]', '[removed]'))
                
                outfile.write('\t'.join(str(x) for x in 
                              [commenter, commentee, sr_name, post_id, controversiality, upvotes,
                               downvotes, score, gilded, created, int(is_reply), author_deleted,
                               text_deleted, body]) + '\n')
                
                comids_to_authors[com['name']] = commenter
            
            if i % 100000 == 0:
                print(i) # Rudimentary progress indicator
                
    return comids_to_authors
        

In [25]:
comment_directory = 'comments_jan2012'
output_comment_text_file = 'reddit_comments_jan2012.txt'
comids_to_authors = parse_comments(srids_to_srnames, postids_to_authors, comment_directory, output_comment_text_file)

Parsing comments...
comments_jan2012/RC_2012-01.bz2: parsing...
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
1160000

In [26]:
len(comids_to_authors)

12994655